Ambient RNA detection
----

Perform ambient RNA detection with SoupX

Environment: `preprocessing.yaml`


## Dependecy notebooks

Input: 

Outpu: 36.372 × 34859

- `02_quality_control.ipynb`

Loads the qc filtered matrix. 

Returns: `adata_ambient_rna_corrected_matrix.h5ad` with ambient RNA genes removed

## Import packages

In [1]:
import os

print(os.environ.get('CONDA_DEFAULT_ENV'))

preprocessing


In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(SoupX)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [4]:
# import logging

# import anndata2ri
# import rpy2.rinterface_lib.callbacks as rcb
# import rpy2.robjects as ro

import scanpy as sc
# import anndata as ad
import numpy as np

from pathlib import Path
from rpy2.robjects import numpy2ri
from rpy2.robjects.conversion import localconverter
from scipy.sparse import csc_matrix

# rcb.logger.setLevel(logging.ERROR)
# ro.pandas2ri.activate()
# anndata2ri.activate()
	
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

rcb.logger.setLevel(logging.ERROR)
pandas2ri.activate()

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Set up paths

In [6]:
import sys
sys.path.insert(0, "../")  # this depends on the notebook depth and must be adapted per notebook

from paths import DATA_DIR, RESULTS_DIR, FIG_DIR
from utils.utils import SAMPLES, VERSION

In [8]:
adata = sc.read_h5ad(Path(DATA_DIR / f"{VERSION}_adata_filtered_counts_matrix_final.h5ad"))
adata

AnnData object with n_obs × n_vars = 36372 × 78932
    obs: 'sample', 'condition', 'area', 'age', 'donor', 'sequencing batch', 'sequencing id', 'sex', 'APOE genotype', 'brain area number', 'running number', 'donor_v2', 'donor_v2_area', 'n_counts', 'log_counts', 'n_genes', 'tech_sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_nuclear', 'log1p_total_counts_nuclear', 'pct_counts_nuclear', 'braak_stage', 'area_abbrev', 'condition_area', 'brainbank', 'brain_area_latin'
    var: 'ensemble_id', 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'mt', 'ribo', 'hb', 'nuclear', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'area_colors', 'brain area latin_colors', 'cell_type_sea_colors', 'condition_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'sample_colors', 'tech_sample_colors', '

In [9]:
adata_pp = adata.copy()
sc.pp.normalize_total(adata_pp, target_sum=1e4)
sc.pp.log1p(adata_pp)

In [10]:
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(
    adata_pp, key_added="soupx_groups", flavor="igraph", n_iterations=2, directed=False
)
# Preprocess variables for SoupX
adata.obs["soupx_groups"] = adata_pp.obs["soupx_groups"]

In [11]:
del adata_pp

In [12]:
cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

SoupX additionally requires the raw gene by cell matrix from `raw_feature_bc_matrix..h5`

In [13]:
adata_raw = sc.read(Path(DATA_DIR / f"{VERSION}_adata_raw_feature_matrix.h5ad"))
adata_raw

AnnData object with n_obs × n_vars = 54957338 × 78932
    obs: 'sample', 'condition', 'area', 'age', 'Braak stage', 'donor', 'sequencing batch', 'Brainbank source', 'sequencing id', 'sex', 'APOE genotype', 'brain area latin', 'brain area number', 'running number'
    var: 'ensemble_id', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'raw'

In [14]:
genes_raw = adata_raw.var_names
cells_raw = adata_raw.obs_names

data_tod = adata_raw.X.T

Check that the gene names are equavalent.

In [15]:
if not (all(genes == genes_raw)):
    # Genes in genes but not in genes_raw
    diff_1 = set(genes) - set(genes_raw)
    
    # Genes in genes_raw but not in genes
    diff_2 = set(genes_raw) - set(genes)
    
    print("In genes but not in genes_raw:", len(diff_1), list(diff_1)[:10])
    print("In genes_raw but not in genes:", len(diff_2), list(diff_2)[:10])

In [16]:
#genes = np.array([g.replace('.', '-') for g in genes])

In [17]:
del adata_raw 

In [18]:
data_csc = data.tocsc()
data_tod_csc = data_tod.tocsc()

# Extract sparse components and cast to correct types
x = data_csc.data.astype(np.float64)
i = data_csc.indices.astype(np.int32)
p = data_csc.indptr.astype(np.int32)
dims = np.array(data_csc.shape, dtype=np.int32)

x_tod = data_tod_csc.data.astype(np.float64)
i_tod = data_tod_csc.indices.astype(np.int32)
p_tod = data_tod_csc.indptr.astype(np.int32)
dims_tod = np.array(data_tod_csc.shape, dtype=np.int32)

with localconverter(ro.default_converter + pandas2ri.converter + numpy2ri.converter):
    ro.globalenv["x"] = x
    ro.globalenv["i"] = i
    ro.globalenv["p"] = p
    ro.globalenv["dims"] = dims

    ro.globalenv["x_tod"] = x_tod
    ro.globalenv["i_tod"] = i_tod
    ro.globalenv["p_tod"] = p_tod
    ro.globalenv["dims_tod"] = dims_tod

    ro.globalenv["genes"] = np.array(genes)
    ro.globalenv["genes_raw"] = np.array(genes_raw)
    ro.globalenv["cells"] = np.array(cells)
    ro.globalenv["cells_raw"] = np.array(cells_raw)
    ro.globalenv["soupx_groups"] = adata.obs["soupx_groups"].to_numpy()

In [19]:
%%R -o out 

library(Matrix)

# Manually coerce types to avoid "array" class errors
x <- as.numeric(x)
i <- as.integer(i)
p <- as.integer(p)
dims <- as.integer(dims)

x_tod <- as.numeric(x_tod)
i_tod <- as.integer(i_tod)
p_tod <- as.integer(p_tod)
dims_tod <- as.integer(dims_tod)

# Reconstruct sparse matrices
data <- new("dgCMatrix",
            Dim = dims,
            x = x,
            i = i,
            p = p)

data_tod <- new("dgCMatrix",
                Dim = dims_tod,
                x = x_tod,
                i = i_tod,
                p = p_tod)

# Assign row and column names
rownames(data) <- genes
colnames(data) <- cells
rownames(data_tod) <- genes_raw
colnames(data_tod) <- cells_raw

# SoupX pipeline
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

soupProf = data.frame(
  row.names = rownames(data),
  est = rowSums(data) / sum(data),
  counts = rowSums(data)
)
sc = setSoupProfile(sc, soupProf)
sc = setClusters(sc, soupx_groups)
sc = autoEstCont(sc, doPlot = FALSE)
out = adjustCounts(sc, roundToInt = TRUE)

Store the SoupX corrected counts. Store old counts in additional layer and overwrite the SoupX counts.

In [20]:
with localconverter(ro.default_converter + pandas2ri.converter + numpy2ri.converter):
    out_py = ro.conversion.rpy2py(ro.globalenv["out"])

x = np.array(out_py.slots["x"])
i = np.array(out_py.slots["i"])
p = np.array(out_py.slots["p"])
shape = tuple(out_py.slots["Dim"])

out_matrix = csc_matrix((x, i, p), shape=shape)

In [21]:
adata.layers["counts"] = adata.X.copy()
adata.layers["soupX_counts"] = out_matrix.T
adata.X = adata.layers["soupX_counts"]

In [22]:
print(f"Total number of genes: {adata.n_vars}")

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print(f"Number of genes after cell filter: {adata.n_vars}")

Total number of genes: 78932
Number of genes after cell filter: 34859


## Save adata

In [23]:
adata

AnnData object with n_obs × n_vars = 36372 × 34859
    obs: 'sample', 'condition', 'area', 'age', 'donor', 'sequencing batch', 'sequencing id', 'sex', 'APOE genotype', 'brain area number', 'running number', 'donor_v2', 'donor_v2_area', 'n_counts', 'log_counts', 'n_genes', 'tech_sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_nuclear', 'log1p_total_counts_nuclear', 'pct_counts_nuclear', 'braak_stage', 'area_abbrev', 'condition_area', 'brainbank', 'brain_area_latin', 'soupx_groups'
    var: 'ensemble_id', 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'mt', 'ribo', 'hb', 'nuclear', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'area_colors', 'brain area latin_colors', 'cell_type_sea_colors', 'condition_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'sample_color

In [24]:
adata.write(Path(DATA_DIR / f"{VERSION}_adata_ambient_rna_corrected_matrix.h5ad"))